In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install -q h5py pyyaml
!pip install -q tf_nightly
!pip install -U tensorflow==2.0.0-rc0


In [0]:
!pip install tensorflow-gpu==2.0.0-rc0
!pip install tensorflow_hub

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
print("Version: ", tf.__version__)
print("Hub version: ", hub.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

In [0]:
mkdir /content/saved_model/variables

In [0]:
mkdir /content/save_model/

In [60]:
!tflite_convert \
--graph_def_file=mtcnn_v1.pb \
--output_file=mtcnn.tflite \
--output_format=TFLITE \
--input_arrays='pnet/input','rnet/input','onet/input' \
--output_arrays='pnet/prob1','pnet/conv4-2/BiasAdd','rnet/prob1','rnet/conv5-2/conv5-2','onet/prob1','onet/conv6-2/conv6-2','onet/conv6-3/conv6-3' \
--inference_type=FLOAT \
--change_concat_input_ranges=false \
--allow_custom_ops \
--saved_model_dir=saved_model \
--input_shapes=1,300,300,3:1,24,24,3:1,48,48,3

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
I1010 11:00:15.044895 140044936685440 saver.py:1503] Saver not created because there are no variables in the graph to restore
2019-10-10 11:00:15.054876: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-10 11:00:15.057977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-10 11:00:15.058710: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-10 11:00:15.058993: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-10 11:00:15.060220: I tensorflow/stream_executor/platform/default

In [50]:
!tflite_convert \
--saved_model_dir=/content/saved_model \
--output_file=mobilenet.tflite

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
I1010 10:10:26.010404 140541953054592 saver.py:1503] Saver not created because there are no variables in the graph to restore
2019-10-10 10:10:26.021548: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-10 10:10:26.024977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-10 10:10:26.025747: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-10 10:10:26.026027: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-10 10:10:26.027480: I tensorflow/stream_executor/platform/default

In [0]:
graph_def_file = "/content/mtcnn.pb"
input_arrays = ["pnet/input","rnet/input","onet/input"]
output_arrays = ["pnet/prob1","pnet/conv4-2/BiasAdd","rnet/prob1","rnet/conv5-2/conv5-2","onet/prob1","onet/conv6-2/conv6-2","onet/conv6-3/conv6-3"]
input_shapes=[1,512,512,3],[1,24,24,3],[1,48,48,3]
converter = tf.lite.TFLiteConverter.from_frozen_graph(graph_def_file, input_arrays, output_arrays, input_shapes)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [42]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="mtcnn.pb")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.uint8)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print('INPUTS: ')
print(input_details)
print('OUTPUTS: ')
print(output_details)

ValueError: ignored

In [0]:
!tflite_convert \
  --output_file=/tmp/foo.tflite \
  --keras_model_file=/content/my_mnist_v2.h5

In [0]:
  --output_arrays=box \
!python freeze_graph.py --input_graph=/content/save_model/saved_model.pbtxt --input_checkpoint=/content/save_model/mtcnn-3000000 --input_binary=false --output_graph=/content/save_model/frozen_graph.pb --output_node_names="the nodes that you want to output e.g. InceptionV3/Predictions/Reshape_1 for Inception V3 "

In [0]:
import tensorflow as tf
from facenet.src.models import inception_resnet_v1
import sys


def main():
    traning_checkpoint = "models/mtcnn-300000.index"
    eval_checkpoint = "model_inference/imagenet_facenet.ckpt"

    data_input = tf.placeholder(name='input', dtype = tf.float32, shape = [None, 160, 160, 3])
    output, _ = inception_resnet_v1.inference(data_input, keep_probability=0.8, phase_train=False,
                                              bottleneck_layer_size=512)
    label_batch = tf.identity(output, name='label_batch')
    embeddings = tf.identity(output, name='embeddings')

    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        saver = tf.train.Saver()
        saver.restore(sess, traning_checkpoint)
        save_path = saver.save(sess, eval_checkpoint)
        print("Model saved in file: % s" % save_path)
        if __name__ == "__main__":
            main()

In [0]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model_file("abc.h5")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [0]:
import tensorflow as tf

graph_def_file = "mtcnn_1.12.pb"
input_arrays = ["input"]
output_arrays = ["box"]
converter = tf.lite.TFLiteConverter.from_frozen_graph(graph_def_file, input_arrays, output_arrays)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)